In [1]:
import pickle
import numpy as np
import time
from sklearn.gaussian_process.kernels import ExpSineSquared, RBF
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
from sklearn.kernel_approximation import RBFSampler
import torch
from sklearn.ensemble import RandomTreesEmbedding, RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from datetime import datetime

c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/final_lucknow_with_lat_long_stationwise_split_train_timestamp_station_names.pkl", "rb") as f:
    train_data = pickle.load(f)

with open("./data/final_lucknow_with_lat_long_stationwise_split_test_timestamp_station_names.pkl", "rb") as f:
    test_data = pickle.load(f)

In [5]:
def break_timestamp(timestamp: str):

    # Return the month, date and hour
    x = re.search(r'-\d{2}-\d{2}', timestamp)
    date = x.group()[-2:]
    month = x.group()[1:3]
    # time = re.search(r'\s+\d{2}', timestamp)
    # hour = time.group()[-2:]

    return date+month

In [6]:
# Return the encoding for cyclic categories

def cyclical_encode(data, max_val):
    return np.sin(2 * np.pi * data/max_val), np.cos(2 * np.pi * data/max_val)

In [7]:
def convert_data_timestamp(data: list):
    # Change the current dataframe and add the time columns
    new_data = []
    for i in range(len(data)):
        # month, date, hour = break_timestamp(str(data[i]['Timestamp']))
        # month_embed, hour_embed = cyclical_encode(int(month), 12.0), cyclical_encode(int(hour), 24.0)
        # date_embed = np.zeros((31, ))
        # date_embed[int(date)-1] = 1
        # date_embed = [(1 if i == int(date) else 0) for i in range(31)]
        date = break_timestamp(str(data[i]['Timestamp']))

        row = {'Image': data[i]['Image'], 'Meteo': data[i]['Meteo'], 'Date': date, 'PM25': data[i]['PM25']}
        new_data.append(row)
        
    return new_data

In [8]:
train_data = convert_data_timestamp(train_data)
test_data = convert_data_timestamp(test_data)

In [11]:
def convert_data(data: list):
    new_data = []

    for i in range(len(data)):
        row = []
        row.extend(data[i]['Meteo'])
        # row.extend(data[i]['Month'])
        row.extend(data[i]['Date'])
        # row.extend(data[i]['Hour'])
        row.append(data[i]['PM25'])

        new_data.append(row)

    new_data = np.array(new_data)
    return new_data


In [12]:
X_train, y_train = convert_data(train_data)[:, :-1], convert_data(train_data)[:, -1]
X_test, y_test = convert_data(test_data)[:, :-1], convert_data(test_data)[:, -1]

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6796, 40) (6813, 40) (6796,) (6813,)


# Random Forest with Latlong

In [13]:
# Transform timestamps data with Random Trees Embedding Model

# rt_model = RandomTreesEmbedding(n_estimators=800,max_depth=2).fit(X)

rt_model = RandomTreesEmbedding(n_estimators=800,max_depth=2).fit(X_train)
data_transformed_train = rt_model.transform(X_train).toarray()
data_transformed_test = rt_model.transform(X_test).toarray()

In [14]:
# # Train and predict with Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=800, max_features="sqrt", min_samples_leaf=2).fit(data_transformed_train, y_train)
y_train_pred_rf = rf_model.predict(data_transformed_train)
y_test_pred_rf = rf_model.predict(data_transformed_test)

In [15]:
print(math.sqrt(mean_squared_error(y_train, y_train_pred_rf)))

10.1883143675483


c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\sklearn\metrics\_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [16]:
print(math.sqrt(mean_squared_error(y_test, y_test_pred_rf)))

29.577715913538242


c:\Users\malay\miniconda3\envs\cs776\lib\site-packages\sklearn\metrics\_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [17]:
from scipy import stats

In [20]:
def eval_stat(y_train_pred, y_train):
    Rsquared = stats.spearmanr(y_train_pred, y_train.ravel())[0]
    pvalue = stats.spearmanr(y_train_pred, y_train.ravel())[1]
    Rsquared_pearson = stats.pearsonr(y_train_pred, y_train.ravel())[0]
    pvalue_pearson = stats.pearsonr(y_train_pred, y_train.ravel())[1]
    return Rsquared, pvalue, Rsquared_pearson, pvalue_pearson

In [3]:
def convert_data(data: list):
    new_data = []

    for i in range(len(data)):
        row = []
        row.extend(data[i]['Meteo'])
        row.append(data[i]['PM25'])

        new_data.append(row)

    new_data = np.array(new_data)
    return new_data

In [4]:
X_train, y_train = convert_data(train_data)[:, :-1], convert_data(train_data)[:, -1]
X_test, y_test = convert_data(test_data)[:, :-1], convert_data(test_data)[:, -1]

In [5]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6796, 5) (6813, 5) (6796,) (6813,)


In [6]:
# Transform timestamps data with Random Trees Embedding Model

# rt_model = RandomTreesEmbedding(n_estimators=800,max_depth=2).fit(X)

rt_model = RandomTreesEmbedding(n_estimators=800,max_depth=2).fit(X_train)
data_transformed_train = rt_model.transform(X_train).toarray()
data_transformed_test = rt_model.transform(X_test).toarray()

In [9]:
# # Train and predict with Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=800, max_features="sqrt", min_samples_leaf=2).fit(data_transformed_train, y_train)
y_train_pred_rf = rf_model.predict(data_transformed_train)
y_test_pred_rf = rf_model.predict(data_transformed_test)

In [10]:
print(math.sqrt(mean_squared_error(y_train, y_train_pred_rf)))

9.766198347873695


In [11]:
print(math.sqrt(mean_squared_error(y_test, y_test_pred_rf)))

25.194905524016303


In [12]:
from scipy import stats

In [13]:
def eval_stat(y_train_pred, y_train):
    Rsquared = stats.spearmanr(y_train_pred, y_train.ravel())[0]
    pvalue = stats.spearmanr(y_train_pred, y_train.ravel())[1]
    Rsquared_pearson = stats.pearsonr(y_train_pred, y_train.ravel())[0]
    pvalue_pearson = stats.pearsonr(y_train_pred, y_train.ravel())[1]
    return Rsquared, pvalue, Rsquared_pearson, pvalue_pearson

In [14]:
print(eval_stat(y_train_pred_rf, y_train))
print(eval_stat(y_test_pred_rf, y_test))

(0.971987356161981, 0.0, 0.9707454560551183, 0.0)
(0.7284786371592858, 0.0, 0.7299734605626739, 0.0)


In [18]:
embed_path = './models/rt_rf_embedding.pkl'
model_path = './models/rt_rf_regressor.pkl'

In [19]:
pickle.dump(rt_model, open(embed_path, 'wb'))
pickle.dump(rf_model, open(model_path, 'wb'))